# PyTorch Implementation of the NRMS Model for `EBNeRD` (RecSys'24 Challenge)
### Course: `02456 Deep Learning` (Fall 2024)  
**Institution:** Technical University of Denmark (DTU)  
**Authors:** Kevin Moore (s204462) and Nico Tananow (s[insert number])

### Acknowledgments  
1. Special thanks to **Johannes Kruse** for his [TensorFlow implementation of the NRMS Model](https://github.com/ebanalyse/ebnerd-benchmark), which greatly supported the development of this PyTorch implementation for the EBNeRD project.  


2. Our implementation is based on the NRMS model described in the paper **["Neural News Recommendation with Multi-Head Self-Attention"](https://aclanthology.org/D19-1671/)** by Wu et al. (2019).

# Implementation

## 1. Importing Dependencies
Import all necessary libraries and modules, including `utils.model` and `utils.helper` for the NRMS model, data preparation, training, and evaluation.

In [61]:
import sys
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # to avoid warnings in transformers
from pathlib import Path
# Get the current directory
current_dir = os.getcwd()
root_dir = os.path.join(os.path.dirname(os.path.dirname(current_dir)), "dtu-02456-deep-learning-ebnerd")
src_dir = os.path.join(root_dir, "src")
# Append the relative path to the utils folder and ebrec src
sys.path.append(os.path.join(current_dir, "utils"))
sys.path.append(src_dir)
from importlib import reload

import torch
import utils.model
reload(utils.model)
from utils.model import (
    NRMSModel
)

import utils.helper
reload(utils.helper)
from utils.helper import (
    HParams,
    load_articles_and_embeddings,
    prepare_training_data,
    prepare_test_data,
    train_model,
    evaluate_model,
    predict_scores,
    save_hparams
)
import datetime

from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.utils._python import rank_predictions_by_score, write_submission_file
from ebrec.utils._constants import DEFAULT_IMPRESSION_ID_COL

### (Optional) Load from checkpoint

In [15]:
LOAD_FROM_CHECKPOINT = False
if LOAD_FROM_CHECKPOINT:
		CHECKPOINT = "2024-12-18T13-58-31" # Change this to the checkpoint you want to load from
		
		import json
		CHECKPOINT_DIR = os.path.join(current_dir, "checkpoints/" + CHECKPOINT)
		INFO_FILE_PATH = os.path.join(CHECKPOINT_DIR, "info.json")

		with open(INFO_FILE_PATH, "r") as json_file:
					data = json.load(json_file)

		# Extract hyperparameters from JSON
		hparams_data = data.get("Hyperparameters", {})

		# Create an HParams instance with the loaded values
		hparams = HParams(**hparams_data)

		# Load the model from the checkpoint
		pth_file = "nrms_checkpoint_1.pth"
		checkpoint = torch.load(os.path.join(CHECKPOINT_DIR, pth_file))
		NRMSModel.load_state_dict(checkpoint['model_state_dict'])

## 2. Setting Hyperparameters
Initialize the hyperparameters for the model

In [62]:
# Setting hyperparameters
hparams = HParams()
hparams.data_fraction = 0.01
hparams.batch_size = 32

DATASPLIT = hparams.datasplit

## 3. Loading and creating **embeddings** for articles

In [63]:
PATH = Path(os.path.join(current_dir, "data"))
print("Loading data from ", PATH, "with datasplit:", DATASPLIT)

# Loading articles and embeddings
article_mapping, word_embeddings = load_articles_and_embeddings(hparams, PATH)

Loading data from  /Users/kevinmoore/Git Repositories/dtu-02456-deep-learning-ebnerd/our_implementation/data with datasplit: ebnerd_small


## 4. Training the Model
- Initialize the NRMS model with preloaded hyperparameters and embeddings. 
- Train the model using the training and validation datasets. 
- Early stopping is applied with a patience parameter of 3 to prevent overfitting.

In [64]:
# Loading Training and Validation data
train_loader, val_loader = prepare_training_data(
    hparams, PATH, DATASPLIT, article_mapping
)

 -> Train samples: 2342
 -> Validation samples: 332


In [65]:
print("Training model with ", hparams)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NRMSModel(hparams, word_embeddings)

CHECKPOINT_DIR = save_hparams(device, hparams)

model = train_model(
    device,
    model,
    train_loader,
    val_loader,
    hparams,
    patience=3,
    checkpoint_dir=CHECKPOINT_DIR,
)

Training model with  
 title_size: 30
 head_num: 20
 head_dim: 20
 attention_hidden_dim: 200
 dropout: 0.2
 batch_size: 32
 verbose: False
 data_fraction: 0.01
 sampling_nratio: 4
 history_size: 20
 epochs: 1
 learning_rate: 0.001
 transformer_model_name: facebookai/xlm-roberta-base
Training information saved to: checkpoints/2024-12-20T15-56-16


Validation: 100%|██████████| 11/11 [00:01<00:00,  6.50batch/s]


Epoch 1/1, Train Loss: 1.5584, Val Loss: 1.5807, Val AUC: 0.5244, Improvement from Previous Epoch: 0.5244
Checkpoint saved to: checkpoints/2024-12-20T15-56-16/nrms_checkpoint_1.pth


## 5. Evaluating the Model
- Load test data
- Evaluate the trained NRMS model on the testing dataset. 
- Print out performance metrics

In [66]:
# Loading Test Data for final evaluation
print("Loading data from ", PATH, "with datasplit:", DATASPLIT)
test_loader, df_test = prepare_test_data(
    hparams, PATH, DATASPLIT, article_mapping
)

Loading data from  /Users/kevinmoore/Git Repositories/dtu-02456-deep-learning-ebnerd/our_implementation/data with datasplit: ebnerd_small
 -> Testing samples: 2446


In [27]:
# # Evaluate model
# metrics = evaluate_model(model, test_loader, device)
# print("\nValidation Metrics:")
# for metric_name, value in metrics.items():
#     print(f"{metric_name}: {value:.4f}")

Testing: 100%|██████████| 77/77 [00:28<00:00,  2.73batch/s]


Validation Metrics:
auc: 0.5865


In [67]:
scores, labels  = predict_scores(model, test_loader, device)

Testing: 100%|██████████| 77/77 [00:28<00:00,  2.70batch/s]


In [68]:
metrics = MetricEvaluator(
    labels=labels,
    predictions=scores,
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

AUC: 100%|███████████████████████████████| 2446/2446 [00:00<00:00, 45678.56it/s]


<MetricEvaluator class>: 
 {
    "auc": 0.5924188372905562,
    "mrr": 0.37210143454708167,
    "ndcg@5": 0.41878617650906275,
    "ndcg@10": 0.48918820031785415
}

# 6. Writing Submission File

In [69]:
ranked_scores = [list(rank_predictions_by_score(score)) for score in scores]

In [70]:
write_submission_file(
    impression_ids=df_test[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=ranked_scores,
    path=os.joinpath(CHECKPOINT_DIR, "predictions.txt"),
    filename_zip=f"{DATASPLIT}_predictions-{model.__class__.__name__}.zip",
)

AttributeError: 'str' object has no attribute 'joinpath'